In [1]:
import os
import requests
import numpy as np
import pandas as pd
from requests.compat import urljoin

# CONSTANTS

IO_DIR = '/Users/himani/workspace/assignment/Smarkets'

BASE_URL = 'http://smarkets.herokuapp.com/'
RELATIVE_URLS = {
    # Shows all users
    'users_bulk': '/users',
    
    # Shows all affiliates
    'affiliates_bulk': '/affiliates',
    
    # Data about betId. Sample betId: 111
    'bet': '/bets/{betId}',
    
    # Data about affiliateId. Sample affiliateId: 8
    'affiliate': '/affiliates/{affiliate_id}',
    
     # Data about userId. Sample userId: 1
    'user': '/users/{userId}',
    
     # All bets by userId
    'user_bets': '/users/{userId}/bets'
}

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
users_url = urljoin(BASE_URL, RELATIVE_URLS['users_bulk'])
users_df = pd.DataFrame(requests.get(users_url).json())
users_df.rename(columns={'id':'user_id'}, inplace=True)

In [3]:
users_df.head()

,affiliate_id,created,user_id,name
0,4,2017-08-15T05:00:50.254547082Z,0,Amelia\tChapman
1,8,2017-08-15T05:00:50.254547784Z,1,Angela\tDowd
2,3,2017-08-15T05:00:50.254548261Z,2,Wanda\tVance
3,1,2017-08-15T05:00:50.254548724Z,3,Ava\tBaker
4,3,2017-08-15T05:00:50.254549194Z,4,Brandon\tQuinn


In [4]:
affiliates_url = urljoin(BASE_URL, RELATIVE_URLS['affiliates_bulk'])
affiliates_df = pd.DataFrame(requests.get(affiliates_url).json())
affiliates_df.rename(columns={'id':'affiliate_id'}, inplace=True)

In [5]:
affiliates_df.head()

,affiliate_id,name,website
0,0,Benjamin\tHamilton,Dazzleverse.com
1,1,Brandon\tSlater,Fliplounge.com
2,2,Julian\tMorrison,Browseify.com
3,3,Evan\tPoole,Playo.com
4,4,Michelle\tParsons,Thoughtspot.com


In [6]:
bets_df = pd.DataFrame(columns = ['id', 'user_id', 'amount', 'percentage_odds', 'timestamp', 'result'])

users_without_bets = []
errors = []

for user_id in list(users_df.user_id):
    user_bets_url = urljoin(BASE_URL, RELATIVE_URLS['user_bets'].format(userId=user_id))
    try:
        response = requests.get(user_bets_url)
        
        # User not found (should not happen). Also, maybe when user has no bets.
        response.raise_for_status()
        
        user_bets_df = pd.DataFrame(response.json())
        
        if len(user_bets_df):  
            # This will handle a user without any bets, if the output is an empty list
            bets_df = pd.concat([bets_df, user_bets_df])
        else:
            users_without_bets.append(user_id)
        
    except Exception as e:
        errors.append(e)
        pass

bets_df.rename(columns={'id':'bet_id'}, inplace=True)

In [7]:
print(len(users_without_bets))
print(len(errors))

0
0


In [8]:
bets_df.head()

,amount,bet_id,percentage_odds,result,timestamp,user_id
0,7.692,306,78,True,2017-08-15T05:00:50.255098631Z,0
1,1.204,309,41,True,2017-08-15T05:00:50.255100795Z,0
2,6.275,481,17,False,2017-08-15T05:00:50.255215107Z,0
3,4.556,658,58,True,2017-08-15T05:00:50.255340828Z,0
4,9.748,868,24,False,2017-08-15T05:00:50.255479307Z,0


In [9]:
# Save data to local for further analysis

users_df.to_csv(os.path.join(IO_DIR, 'users.csv'))
affiliates_df.to_csv(os.path.join(IO_DIR, 'affiliates.csv'))
bets_df.to_csv(os.path.join(IO_DIR, 'bets.csv'))

In [2]:
# users_df = pd.read_csv(os.path.join(IO_DIR, 'users.csv'))
# affiliates_df = pd.read_csv(os.path.join(IO_DIR, 'affiliates.csv'))
# bets_df = pd.read_csv(os.path.join(IO_DIR, 'bets.csv'))

## 1. Find the affiliate with the maximum number of users.

In [8]:
users_df.groupby('affiliate_id').aggregate('count')['user_id'].idxmax()

1

*******************************************************************************************************************


## 2. Find the amount won by users coming through the top 3 affiliates - by user_count.


In [9]:
top_3_affiliates = list(users_df.groupby('affiliate_id').aggregate('count')['user_id'].sort_values(ascending=False)[:3].index)

In [10]:
users_of_top_3_affiliates = list(users_df[users_df['affiliate_id'].isin(top_3_affiliates)]['user_id'])
len(users_of_top_3_affiliates)

164

In [11]:
bets_df[(bets_df['user_id'].isin(users_of_top_3_affiliates)) & (bets_df['result']==True)]['amount'].sum()

4432.8805185469992

## 3. What is the percentage of users who have won 2 or more bets with low odds - say 25%.


In [14]:
users_won_low_odds_df = bets_df[(bets_df['result']==True) & (bets_df['percentage_odds'] < 25)].groupby('user_id').aggregate('count')
users_won_low_odds = users_won_low_odds_df[users_won_low_odds_df['bet_id'] >= 2].index.tolist()

In [15]:
print(len(users_won_low_odds)*100/len(users_df))

2.8
